# ingest.stores.dense

> vector database for question-answering and other tasks

In [ ]:
# | default_exp ingest.stores.dense

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import os
import os.path
from typing import List, Optional, Callable, Dict
from tqdm import tqdm

from onprem.utils import get_datadir, DEFAULT_DB
from onprem.ingest.base import batchify_chunks, process_folder, does_vectorstore_exist, VectorStore
from onprem.ingest.base import DEFAULT_CHUNK_SIZE, DEFAULT_CHUNK_OVERLAP, TABLE_CHUNK_SIZE, CHROMA_MAX
from langchain_chroma import Chroma
import chromadb
from chromadb.config import Settings

os.environ["TOKENIZERS_PARALLELISM"] = "0"
COLLECTION_NAME = "onprem_chroma"
CHROMA_MAX = 41000



class DenseStore(VectorStore):
    def __init__(
        self,
        persist_directory: Optional[str] = None,
        **kwargs
    ):
        """
        Ingests all documents in `source_folder` (previously-ingested documents are ignored)

        **Args**:

          - *persist_directory*: Path to vector database (created if it doesn't exist).
                                 Default is `onprem_data/vectordb` in user's home directory.
          - *embedding_model*: name of sentence-transformers model
          - *embedding_model_kwargs*: arguments to embedding model (e.g., `{device':'cpu'}`). If None, GPU used if available.
          - *embedding_encode_kwargs*: arguments to encode method of
                                       embedding model (e.g., `{'normalize_embeddings': False}`).


        **Returns**: `None`
        """
        self.persist_directory = persist_directory or os.path.join(
            get_datadir(), DEFAULT_DB
        )
        self.init_embedding_model(**kwargs) # stored in self.embeddings

        self.chroma_settings = Settings(
            persist_directory=self.persist_directory, anonymized_telemetry=False
        )
        self.chroma_client = chromadb.PersistentClient(
            settings=self.chroma_settings, path=self.persist_directory
        )
        return


    def get_db(self):
        """
        Returns an instance to the `langchain_chroma.Chroma` instance
        """
        db = Chroma(
            persist_directory=self.persist_directory,
            embedding_function=self.embeddings,
            client_settings=self.chroma_settings,
            client=self.chroma_client,
            collection_metadata={"hnsw:space": "cosine"},
            collection_name=COLLECTION_NAME,
        )
        return db if does_vectorstore_exist(db) else None


    def exists(self):
        return self.get_db() is not None


    def add_documents(self, documents, batch_size:int=CHROMA_MAX):
        """
        Stores instances of `langchain_core.documents.base.Document` in vectordb
        """
        if not documents:
            return
        db = self.get_db()
        if db:
            print("Creating embeddings. May take some minutes...")
            chunk_batches, total_chunks = batchify_chunks(documents, batch_size=batch_size)
            for lst in tqdm(chunk_batches, total=total_chunks):
                db.add_documents(lst)
        else:
            chunk_batches, total_chunks = batchify_chunks(documents, batch_size)
            print("Creating embeddings. May take some minutes...")
            db = None

            for lst in tqdm(chunk_batches, total=total_chunks):
                if not db:
                    db = Chroma.from_documents(
                        lst,
                        self.embeddings,
                        persist_directory=self.persist_directory,
                        client_settings=self.chroma_settings,
                        client=self.chroma_client,
                        collection_metadata={"hnsw:space": "cosine"},
                        collection_name=COLLECTION_NAME,
                    )
                else:
                    db.add_documents(lst)
        return


    def remove_document(self, id_to_delete):
        """
        Remove a single document with ID, `id_to_delete`.
        """
        if not self.exists(): return
        self.get_db().delete(ids=[id_to_delete])
        return

    def get_all_docs(self):
        """
        Returns a list of files previously added to vector database (typically via `LLM.ingest`)
        """
        if not self.exists(): return []
        return set([d['source'] for d in self.get_db().get()['metadatas']])


    def get_doc(self, id):
        """
        Retrieve a record by ID
        """
        if not self.exists(): return None
        return self.get_db().get(ids=[id])



    def get_size(self):
        """
        Get total number of records
        """
        if not self.exists(): return 0
        return self.get_db().count()

    
    def erase(self, confirm=True):
        """
        Resets collection and removes and stored documents
        """
        if not self.exists(): return True
        shall = True
        if confirm:
            msg = (
                f"You are about to remove all documents from the vector database."
                + f"(Original documents on file system will remain.) Are you sure?"
            )
            shall = input("%s (Y/n) " % msg) == "Y"
        if shall:
            self.get_db().reset_collection()
            return True
        return False


    def query(self,
              query:str, # query string
              k:int = 4, # max number of results to return
              filters:Optional[Dict[str, str]] = None, # filter sources by metadata values using Chroma metadata syntax (e.g., {'table':True})
              where_document:Optional[Dict[str, str]] = None, # filter sources by document content in Chroma syntax (e.g., {"$contains": "Canada"})
              **kwargs):
        """
        Perform a semantic search of the vector DB
        """
        if not self.exists(): return []
        db = self.get_db()
        results = db.similarity_search_with_score(query, 
                                                  filter=filters,
                                                  where_document=where_document,
                                                  k = k, **kwargs)
        return results

    def semantic_search(self, *args, **kwargs):
        """
        Semantic search is equivalent to queries in this class
        """
        return self.query(*args, **kwargs)

    


    def ingest(
        self,
        source_directory: str, # path to folder containing document store
        chunk_size: int = DEFAULT_CHUNK_SIZE, # text is split to this many characters by [langchain.text_splitter.RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)
        chunk_overlap: int = DEFAULT_CHUNK_OVERLAP, # character overlap between chunks in `langchain.text_splitter.RecursiveCharacterTextSplitter`
        ignore_fn:Optional[Callable] = None, # Optional function that accepts the file path (including file name) as input and returns `True` if file path should not be ingested.
        batch_size:int=CHROMA_MAX, # batch size used when processing documents
        **kwargs
    ) -> None:
        """
        Ingests all documents in `source_directory` (previously-ingested documents are
        ignored). When retrieved, the
        [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html)
        objects will each have a `metadata` dict with the absolute path to the file
        in `metadata["source"]`.
        Extra kwargs fed to `ingest.load_single_document`.
        """

        if not os.path.exists(source_directory):
            raise ValueError("The source_directory does not exist.")
        elif os.path.isfile(source_directory):
            raise ValueError(
                "The source_directory argument must be a folder, not a file."
            )
        texts = None
        db = self.get_db()
        if db:
            # Update and store locally vectorstore
            print(f"Appending to existing vectorstore at {self.persist_directory}")
            collection = db.get()
            ignored_files=[ metadata["source"] for metadata in collection["metadatas"]]
        else:
            print(f"Creating new vectorstore at {self.persist_directory}")
            ignored_files = []

        texts = process_folder(
            source_directory,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            ignored_files=ignored_files,
            ignore_fn=ignore_fn,
            batch_size=batch_size,
            **kwargs

        )

        texts = list(texts)
        print(f"Split into {len(texts)} chunks of text (max. {chunk_size} chars each for text; max. {TABLE_CHUNK_SIZE} chars for tables)")

        self.add_documents(texts, batch_size=batch_size)

        if texts:
            print(
                "Ingestion complete! You can now query your documents using the LLM.ask or LLM.chat methods"
            )
        db = None
        return

In [ ]:
show_doc(DenseStore.add_documents)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L69){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.add_documents

>      DenseStore.add_documents (documents, batch_size:int=41000)

*Stores instances of `langchain_core.documents.base.Document` in vectordb*

In [ ]:
show_doc(DenseStore.get_all_docs)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L115){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.get_all_docs

>      DenseStore.get_all_docs ()

*Returns a list of files previously added to vector database (typically via `LLM.ingest`)*

In [ ]:
show_doc(DenseStore.get_size)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L122){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.get_size

>      DenseStore.get_size ()

*Get total number of records*

In [ ]:
show_doc(DenseStore.erase)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L129){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.erase

>      DenseStore.erase (confirm=True)

*Resets collection and removes and stored documents*

In [ ]:
show_doc(DenseStore.query)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L146){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.query

>      DenseStore.query (query:str, k:int=4,
>                        filters:Optional[Dict[str,str]]=None,
>                        where_document:Optional[Dict[str,str]]=None, **kwargs)

*Perform a semantic search of the vector DB*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| query | str |  | query string |
| k | int | 4 | max number of results to return |
| filters | Optional | None | filter sources by metadata values using Chroma metadata syntax (e.g., {'table':True}) |
| where_document | Optional | None | filter sources by document content in Chroma syntax (e.g., {"$contains": "Canada"}) |
| kwargs | VAR_KEYWORD |  |  |

In [ ]:
show_doc(DenseStore.get_embedding_model)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L177){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.get_embedding_model

>      DenseStore.get_embedding_model ()

*Returns an instance to the `langchain_huggingface.HuggingFaceEmbeddings` instance*

In [ ]:
show_doc(DenseStore.get_db)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L163){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.get_db

>      DenseStore.get_db ()

*Returns an instance to the `langchain_chroma.Chroma` instance*

In [ ]:
show_doc(DenseStore.ingest)

---

[source](https://github.com/amaiya/onprem/blob/master/onprem/ingest/stores/dense.py#L184){target="_blank" style="float:right; font-size:smaller"}

### DenseStore.ingest

>      DenseStore.ingest (source_directory:str, chunk_size:int=500,
>                         chunk_overlap:int=50,
>                         ignore_fn:Optional[Callable]=None,
>                         batch_size:int=41000, **kwargs)

*Ingests all documents in `source_directory` (previously-ingested documents are
ignored). When retrieved, the
[Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html)
objects will each have a `metadata` dict with the absolute path to the file
in `metadata["source"]`.
Extra kwargs fed to `ingest.load_single_document`.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| source_directory | str |  | path to folder containing document store |
| chunk_size | int | 500 | text is split to this many characters by [langchain.text_splitter.RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html) |
| chunk_overlap | int | 50 | character overlap between chunks in `langchain.text_splitter.RecursiveCharacterTextSplitter` |
| ignore_fn | Optional | None | Optional function that accepts the file path (including file name) as input and returns `True` if file path should not be ingested. |
| batch_size | int | 41000 | batch size used when processing documents |
| kwargs | VAR_KEYWORD |  |  |
| **Returns** | **None** |  |  |

In [ ]:
# | notest
store = DenseStore()
store.ingest("tests/sample_data/ktrain_paper/")

Creating new vectorstore at /home/amaiya/onprem_data/vectordb
Loading documents from tests/sample_data/ktrain_paper/


Processing and chunking 6 new documents: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 899.87it/s]


Split into 41 chunks of text (max. 500 chars each for text; max. 2000 chars for tables)
Creating embeddings. May take some minutes...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.09it/s]

Ingestion complete! You can now query your documents using the LLM.ask or LLM.chat methods


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()